In [1]:
!pip install xmltodict
!pip install pandas

In [2]:
import requests
import re
from datetime import datetime
from time import sleep
from math import ceil
from random import randint
import xmltodict  # Elementtree based
import json
import pandas as pd

In [81]:
#(1/3) 법령 목록 조회 및 Raw-Data 저장

today = datetime.today().strftime('%Y%m%d') # 파일명 등에 사용할 날짜 지정
directory = '01_현행법령목록' # 저장 위치 디렉터리명 지정

uri_OC = '<your-openapi-id>'  #(필수)신청 및 승인된 ID
uri_target = 'law'  #(필수)law
uri_type = 'XML'  #(필수)HTML or XML
uri_display = 100  #요청시 표출되는 법령 목록 수
uri_sort = 'lasc'  #요청시 표출되는 법령명 오름차순(기본:lasc)
uri_page = 1  #총 페이지를 조회하기 위해 지정할 최초의 페이지 번호

In [4]:
def createURIstring(OC, target, type, page, display, sort):
    uri = f'https://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target={target}&type={type}&page={page}&display={display}&sort={sort}'
    return uri

In [7]:
def getTotalPageNum():
    uri = createURIstring(uri_OC, uri_target, uri_type, uri_page, uri_display, uri_sort)
    r = requests.get(uri)
    p = re.compile(r'<totalCnt>([0-9]+)</totalCnt>')
    try:
        total_item_num = int(p.findall(r.text)[0])
        total_page_num = ceil(total_item_num / uri_display)
        return total_page_num, total_item_num
    except IndexError:
        print('국가법령정보 공동활용 OPEN API의 할당 ID 및 접근가능 IP를 체크해주세요.')

In [8]:
total_page_num, total_item_num = getTotalPageNum()

In [10]:
%%capture cap1
%%time

print(  '-'*50, '\n',
        'Start : ', datetime.today().strftime('%Y-%m-%d %H:%M:%S'), '\n',
        'Total Item Number : ', total_item_num, '\n',
        'Total Page Number : ', total_page_num, '\n',
        '-'*50, '\n',)

pages_completed = []
text_merged = ''

for page_num in range(1, total_page_num + 1):

    uri = createURIstring(uri_OC, uri_target, uri_type, page_num, uri_display, uri_sort)
    r = requests.get(uri)

    text_merged = text_merged + r.text
    print(f'%s : %04d-%04d pages saved.' % (uri, page_num, total_page_num))
    sleep( randint(5,10) )

print( '-'*50, '\n', )

In [11]:
# p = re.compile('<page>[0-9]+</page>')
# p.findall(text_merged)

In [12]:
# text_merged

In [13]:
# XML 파일을 통합하여 하나로 저장함.
with open(f'{directory}/{directory}_{today}.txt', 'w') as f:
    f.write(text_merged)
    f.close()

In [14]:
#(2/3) 법령 목록 Raw-Data 저장 파일을 변환 : XML string to JSON
f = open(f'./{directory}/{directory}_{today}.txt', 'r').read()
# f[:1000]

In [15]:
# p1 = re.compile(r'''<[?]xml version="1.0" encoding="UTF-8"[?]>''')
# p1.findall(f)

In [16]:
header = '<?xml version="1.0" encoding="UTF-8"?>'
f_replaced = f.replace(header, '')
f_replaced = header + '<root>' + f_replaced + '</root>' # Trouble shooting : https://www.codeproject.com/questions/1059071/error-parsing-xml-junk-after-document-element

# f_replaced[:1000]

In [17]:
with open(f'./{directory}/{directory}_{today}.json', 'w', encoding="utf-8") as outfile:
    json.dump(xmltodict.parse(f_replaced), outfile)

In [18]:
#(3/3) 법령 목록 JSON 파일을 기준으로 세부 법령 내용 요청

with open(f'./{directory}/{directory}_{today}.json', 'r', encoding="utf-8") as json_data:
    data = json.loads(json_data.read())

In [19]:
print(
    'Data Type : %s' % type(data), '\n',
    'Keys of dict : %s' % data.keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['root']) 



In [20]:
print(
    'Wrapped Pages DType : %s' % type( data['root']['LawSearch'] ), '\n',
    'Total Pages : %03d' % len( data['root']['LawSearch'] ), '\n',
)

Wrapped Pages DType : <class 'list'> 
 Total Pages : 048 



In [21]:
print(
    'Page Return Status', '-'*50, '\n',
    'Data Type of Each Page : %s' % type( data['root']['LawSearch'][0] ), '\n',
    'N-Keys of Each Page : %03d' % len( data['root']['LawSearch'][0] ), '\n',
    'Keys of Each Page : %s' % data['root']['LawSearch'][0].keys(), '\n'
)

Page Return Status -------------------------------------------------- 
 Data Type of Each Page : <class 'dict'> 
 N-Keys of Each Page : 009 
 Keys of Each Page : dict_keys(['target', '키워드', 'section', 'totalCnt', 'page', 'numOfRows', 'resultCode', 'resultMsg', 'law']) 



In [22]:
for key in data['root']['LawSearch'][0].keys():
    print(key, data['root']['LawSearch'][0][key])

target law
키워드 *
section lawNm
totalCnt 4754
page 1
numOfRows 100
resultCode 00
resultMsg success
law [{'@id': '1', '법령일련번호': '180715', '현행연혁코드': '현행', '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률', '법령약칭명': '10ㆍ27법난법', '법령ID': '010719', '공포일자': '20160203', '공포번호': '13954', '제개정구분명': '일부개정', '소관부처코드': '1371000', '소관부처명': '문화체육관광부', '법령구분명': '법률', '시행일자': '20160630', '자법타법여부': None, '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=180715&type=HTML&mobileYn=&efYd=20160630'}, {'@id': '2', '법령일련번호': '183817', '현행연혁코드': '현행', '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령', '법령약칭명': '10ㆍ27법난법시행령', '법령ID': '010831', '공포일자': '20160608', '공포번호': '27211', '제개정구분명': '일부개정', '소관부처코드': '1371000', '소관부처명': '문화체육관광부', '법령구분명': '대통령령', '시행일자': '20160630', '자법타법여부': None, '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=183817&type=HTML&mobileYn=&efYd=20160630'}, {'@id': '3', '법령일련번호': '200188', '현행연혁코드': '현행', '법령명한글': '119구조ㆍ구급에 관한 법률', '법령약칭명': '119법', '법령ID': '011349', '공포일자': '20171226', '공포번호': '1

In [23]:
data['root']['LawSearch'][0]['law']

[{'@id': '1',
  '법령일련번호': '180715',
  '현행연혁코드': '현행',
  '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률',
  '법령약칭명': '10ㆍ27법난법',
  '법령ID': '010719',
  '공포일자': '20160203',
  '공포번호': '13954',
  '제개정구분명': '일부개정',
  '소관부처코드': '1371000',
  '소관부처명': '문화체육관광부',
  '법령구분명': '법률',
  '시행일자': '20160630',
  '자법타법여부': None,
  '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=180715&type=HTML&mobileYn=&efYd=20160630'},
 {'@id': '2',
  '법령일련번호': '183817',
  '현행연혁코드': '현행',
  '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령',
  '법령약칭명': '10ㆍ27법난법시행령',
  '법령ID': '010831',
  '공포일자': '20160608',
  '공포번호': '27211',
  '제개정구분명': '일부개정',
  '소관부처코드': '1371000',
  '소관부처명': '문화체육관광부',
  '법령구분명': '대통령령',
  '시행일자': '20160630',
  '자법타법여부': None,
  '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=183817&type=HTML&mobileYn=&efYd=20160630'},
 {'@id': '3',
  '법령일련번호': '200188',
  '현행연혁코드': '현행',
  '법령명한글': '119구조ㆍ구급에 관한 법률',
  '법령약칭명': '119법',
  '법령ID': '011349',
  '공포일자': '20171226',
  '공포번호': '15298',
  '제개정구분명': '일부개정',
  '

In [24]:
pd.DataFrame(
    data['root']['LawSearch'][0]['law'],
)

,@id,공포번호,공포일자,법령ID,법령구분명,법령명한글,법령상세링크,법령약칭명,법령일련번호,소관부처명,소관부처코드,시행일자,자법타법여부,제개정구분명,현행연혁코드
0,1,13954,20160203,010719,법률,10ㆍ27법난 피해자의 명예회복 등에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,10ㆍ27법난법,180715,문화체육관광부,1371000,20160630,None,일부개정,현행
1,2,27211,20160608,010831,대통령령,10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=18...,10ㆍ27법난법시행령,183817,문화체육관광부,1371000,20160630,None,일부개정,현행
2,3,15298,20171226,011349,법률,119구조ㆍ구급에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,119법,200188,소방청,1661000,20180627,None,일부개정,현행
3,4,28216,20170726,011452,대통령령,119구조ㆍ구급에 관한 법률 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=19...,119법시행령,195947,소방청,1661000,20170726,None,타법개정,현행
4,5,00002,20170726,011457,행정안전부령,119구조ㆍ구급에 관한 법률 시행규칙,/DRF/lawService.do?OC=tooget&target=law&MST=19...,119법시행규칙,196225,소방청,1661000,20170726,None,타법개정,현행
5,6,04401,19691204,004546,대통령령,1945년이후종전의규정에의한학교졸업자자격인정령,/DRF/lawService.do?OC=tooget&target=law&MST=24...,None,24144,교육부,1342000,19691204,None,전부개정,현행
6,7,10820,20110714,009678,법률,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=11...,1959이전군퇴직금법,114766,국방부,1290000,20120101,None,일부개정,현행
7,8,27083,20160405,009990,대통령령,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=18...,1959이전군퇴직금법시행령,182269,국방부,1290000,20160405,None,타법개정,현행
8,9,04101,19890329,001348,법률,1980년해직공무원의보상등에관한특별조치법,/DRF/lawService.do?OC=tooget&target=law&MST=32...,1980해직자보상법,3274,인사혁신처,1760000,19890329,None,제정,현행
9,10,29181,20180918,004549,대통령령,1980년해직공무원의보상등에관한특별조치법시행령,/DRF/lawService.do?OC=tooget&target=law&MST=20...,1980해직자보상법시행령,204611,인사혁신처,1760000,20180921,None,타법개정,현행


In [28]:
%%capture cap2
%%time

df_list_of_laws = pd.DataFrame()

for page_idx in range(0, len(data['root']['LawSearch'])):
    
    df_list_of_laws = pd.concat([
                            df_list_of_laws,
                            pd.DataFrame( data['root']['LawSearch'][page_idx]['law'] ),
                      ])
    
    print('Return %s, Dataframe of Page Number %s (%s) concatenated' % (
                                                        data['root']['LawSearch'][page_idx]['resultMsg'],
                                                        data['root']['LawSearch'][page_idx]['page'],
                                                        data['root']['LawSearch'][page_idx]['totalCnt'],)
    )

In [29]:
df_list_of_laws.to_csv(f'./{directory}/{directory}_{today}.csv', encoding='utf-8', index=False)

In [30]:
# 조회 내용 로그를 저장함.
with open(f'{directory}/{directory}_{today}_result.txt', 'w') as f:
    f.write(cap1.stdout)
    f.write(cap2.stdout)
    f.close()

In [40]:
df_list_of_laws['법령구분명'].unique()

array(['법률', '대통령령', '행정안전부령', '총리령', '대법원규칙', '환경부령', '여성가족부령',
       '농림축산식품부령', '중앙선거관리위원회규칙', '국토교통부령', '보건복지부령', '감사원규칙', '기획재정부령',
       '통일부령', '건설교통부령', '고용노동부령', '법무부령', '문화체육관광부령', '헌법재판소규칙',
       '산업통상자원부령', '행정자치부령', '교육부령', '국방부령', '재정경제부령', '국회규칙', '지식경제부령',
       '해양수산부령', '과학기술정보통신부령', '교육인적자원부령', '국토해양부령', '농림수산식품부령', '외교부령',
       '교육과학기술부령', '노동부령', '중소벤처기업부령', '헌법', '보건복지가족부령', '상공부령', '경제기획원령',
       '대통령긴급명령', '각령'], dtype=object)

In [140]:
list_of_industry_law = df_list_of_laws[ (df_list_of_laws['법령명한글'].str.contains('금융')) & (df_list_of_laws['법령구분명'].str.contains('법')) ].reset_index(drop=True)
list_of_industry_law

,@id,공포번호,공포일자,법령ID,법령구분명,법령명한글,법령상세링크,법령약칭명,법령일련번호,소관부처명,소관부처코드,시행일자,자법타법여부,제개정구분명,현행연혁코드
0,963,15287,20171226,000507,법률,국제금융기구에의 가입조치에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,국제금융기구가입법,200177,기획재정부,1051000,20171226,None,일부개정,현행
1,1181,15018,20171031,000534,법률,금융산업의 구조개선에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=19...,금융산업구조개선법,198538,금융위원회,1160100,20171031,None,일부개정,현행
2,1183,14242,20160529,000549,법률,금융실명거래 및 비밀보장에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,금융실명법,183676,금융위원회,1160100,20161201,None,타법개정,현행
3,1190,15412,20180221,000552,법률,금융위원회의 설치 등에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,금융위원회법,202322,금융위원회,1160100,20180822,None,일부개정,현행
4,1192,11069,20110930,010589,법률,금융중심지의 조성과 발전에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=11...,금융중심지법,117424,금융위원회,1160100,20120401,None,일부개정,현행
5,1194,14817,20170418,009374,법률,금융지주회사법,/DRF/lawService.do?OC=tooget&target=law&MST=19...,None,193456,금융위원회,1160100,20171019,None,일부개정,현행
6,1196,14242,20160529,000533,법률,금융회사부실자산 등의 효율적 처리 및 한국자산관리공사의 설립에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,자산관리공사법,183678,금융위원회,1160100,20161201,None,타법개정,현행
7,1198,14818,20170418,012338,법률,금융회사의 지배구조에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=19...,금융사지배구조법,193458,금융위원회,1160100,20171019,None,일부개정,현행
8,1487,14820,20170418,009348,법률,대부업 등의 등록 및 금융이용자 보호에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=19...,대부업법,193460,금융위원회,1160100,20171019,None,일부개정,현행
9,2376,15020,20171031,012513,법률,서민의 금융생활 지원에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=19...,서민금융법,198540,금융위원회,1160100,20171031,None,일부개정,현행


In [137]:
def createURIwithLawIdquery(OC, target, type, lawID):
    uri = f'http://www.law.go.kr/DRF/lawService.do?OC={OC}&target={target}&type={type}&ID={lawID}'
    print(uri, '\n')
    return uri

In [138]:
def saveEachLawContent(lawName, lawID):
    uri = createURIwithLawIdquery( uri_OC, uri_target, uri_type, lawID )

    r = requests.get(uri)
    text = r.text
    
    # XML 파일을 저장함.
    with open(f'{directory}/{directory}_{lawName}_{lawID}_{today}.txt', 'w') as f:
        f.write(text)
        f.close()
    print(f'{directory}/{directory}_{lawName}_{lawID}_{today}.txt', ' saved.')
    
    # 법령 목록 Raw-Data 저장 파일을 변환 : XML string to JSON
    f = open(f'./{directory}/{directory}_{lawName}_{lawID}_{today}.txt', 'r').read()

    header = '<?xml version="1.0" encoding="UTF-8"?>'
    f_replaced = f.replace(header, '')
    f_replaced = header + '<root>' + f_replaced + '</root>' # Trouble shooting : https://www.codeproject.com/questions/1059071/error-parsing-xml-junk-after-document-element

    with open(f'./{directory}/{directory}_{lawName}_{lawID}_{today}.json', 'w', encoding="utf-8") as outfile:
        json.dump(xmltodict.parse(f_replaced), outfile)
    print(f'{directory}/{directory}_{lawName}_{lawID}_{today}.json', ' saved.')

In [139]:
for i in range(len(list_of_industry_law)):

    saveEachLawContent(
        str( df_list_of_industry_law.loc[i]['법령명한글'] ),
        str( df_list_of_industry_law.loc[i]['법령ID'] ),
    )

http://www.law.go.kr/DRF/lawService.do?OC=tooget&target=law&type=XML&ID=000507 

01_현행법령목록/01_현행법령목록_국제금융기구에의 가입조치에 관한 법률_000507_20181030.txt  saved.
01_현행법령목록/01_현행법령목록_국제금융기구에의 가입조치에 관한 법률_000507_20181030.json  saved.
http://www.law.go.kr/DRF/lawService.do?OC=tooget&target=law&type=XML&ID=000534 

01_현행법령목록/01_현행법령목록_금융산업의 구조개선에 관한 법률_000534_20181030.txt  saved.
01_현행법령목록/01_현행법령목록_금융산업의 구조개선에 관한 법률_000534_20181030.json  saved.
http://www.law.go.kr/DRF/lawService.do?OC=tooget&target=law&type=XML&ID=000549 

01_현행법령목록/01_현행법령목록_금융실명거래 및 비밀보장에 관한 법률_000549_20181030.txt  saved.
01_현행법령목록/01_현행법령목록_금융실명거래 및 비밀보장에 관한 법률_000549_20181030.json  saved.
http://www.law.go.kr/DRF/lawService.do?OC=tooget&target=law&type=XML&ID=000552 

01_현행법령목록/01_현행법령목록_금융위원회의 설치 등에 관한 법률_000552_20181030.txt  saved.
01_현행법령목록/01_현행법령목록_금융위원회의 설치 등에 관한 법률_000552_20181030.json  saved.
http://www.law.go.kr/DRF/lawService.do?OC=tooget&target=law&type=XML&ID=010589 

01_현행법령목록/01_현행법령목록_금융중심지의 조성과 발전에 관한 법률_010589_2

In [141]:
def readJSONtoDataframe(json_file_path):

    #법령 JSON 파일을 기준으로 세부 법령 내용 요청
    with open(json_file_path, 'r', encoding="utf-8") as json_data:
        data = json.loads(json_data.read())
    
    return data

In [143]:
json_010513 = readJSONtoDataframe( f'./{directory}/{directory}_자본시장과 금융투자업에 관한 법률_010513_{today}.json' )

In [144]:
print(
    'Data Type : %s' % type(json_010513), '\n',
    'Keys of dict : %s' % json_010513.keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['root']) 



In [145]:
print(
    'Data Type : %s' % type(json_010513['root']), '\n',
    'Keys of dict : %s' % json_010513['root'].keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['법령']) 



In [146]:
print(
    'Data Type : %s' % type(json_010513['root']['법령']), '\n',
    'Keys of dict : %s' % json_010513['root']['법령'].keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['@법령키', '기본정보', '조문', '부칙', '별표', '개정문', '제개정이유']) 



In [212]:
contents_list = []

def extractJsonContent(docu):
    if type(docu) == list:
        for element in docu:
            extractJsonContent(element)
    elif type(docu) == dict:
        key_list = list(docu.keys())
        for key in key_list:
            extractJsonContent(docu[key])
    else:
        contents_list.append(docu)

In [213]:
extractJsonContent(json_010513['root']['법령'])